In [6]:
#calling the libraries#callin 
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from subprocess import check_output

In [11]:
#Calling the datasets
books= pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')
book_tags = pd.read_csv('book_tags.csv')
tags = pd.read_csv('tags.csv')
to_read = pd.read_csv('to_read.csv')

# Machine Learning

### Choosing the necessary information for books dataset

In [12]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


### We want to find unique numbers of users id and books id

In [9]:
n_users = ratings.user_id.unique().shape[0]
n_books = ratings.book_id.unique().shape[0]
print ("Number of users = " + str(n_users) + " | Number of books = " + str(n_books))

Number of users = 53424 | Number of books = 10000


### Pivot Rating 

In [14]:
Ratings = ratings.pivot(index = 'user_id', columns ='book_id', values = 'rating').fillna(0)
Ratings.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,4.0,4.0,0.0,4.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### We need to de-normalize the data and and convert it from a dataframe to a numpy array. 

In [15]:
R = Ratings.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [16]:
sparsity = round(1.0 - len(ratings) / float(n_users * n_books), 3)
print ("The sparsity level of Books dataset is " +  str(sparsity * 100) + "%")

The sparsity level of Books dataset is 98.9%


### SVD Setup

In [17]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 50)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

### With the predictions matrix for every user, we can build a function to recommend books for any user.

In [22]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

book_id,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,0.566186,0.200649,0.239258,2.980974,2.009089,0.724415,0.246338,1.398146,0.363926,3.986432,...,0.054892,-0.000480,-0.007056,-0.004622,0.010961,0.014349,0.004064,0.000255,-0.003378,-0.007225
1,0.078189,4.485378,0.169751,2.444546,1.961955,0.644662,0.636027,1.709605,1.040235,2.794773,...,0.000463,0.003780,0.008717,-0.012558,-0.002044,0.010542,0.036487,0.003754,0.036672,0.038280
2,-0.031903,0.078673,-0.135178,1.637163,0.927012,-0.180742,0.209149,0.956741,-0.094428,0.397977,...,0.014760,0.008304,0.008942,0.009337,0.013517,0.003029,0.010829,0.005794,0.013871,0.011592
3,0.018876,4.534593,-0.041104,5.131651,3.153025,-0.854889,2.633736,3.787958,1.953813,2.521620,...,0.007307,-0.043098,-0.023740,0.018297,0.012278,0.010639,-0.008373,-0.000970,-0.006310,0.000703
4,-0.239790,-0.111355,-0.088477,0.123276,0.199847,1.509024,-0.066551,0.099762,0.102077,0.108069,...,0.024209,0.034106,0.025075,0.014417,0.017412,0.027896,0.021651,0.025852,0.032679,0.017620


### We will write function with the highest predicted rating that the specified user hasn't already rated.

In [23]:
def recommend_books(predictions, userID, books, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the book information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(books, how = 'left', left_on = 'book_id', right_on = 'book_id').
                     sort_values(['rating'], ascending=False)
                 )

    print ("User {0} has already rated {1} books.".format(userID, user_full.shape[0]))
    print ("Recommending highest {0} predicted ratings books not already rated.".format(num_recommendations))
    
    # Recommend the highest predicted rating books that the user hasn't seen yet.
    recommendations = (books[~books['book_id'].isin(user_full['book_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'book_id',
               right_on = 'book_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

### We will try to recommend 20 books for user_id 20:

In [24]:
already_rated, predictions = recommend_books(preds, 20, books, ratings, 20)

User 20 has already rated 90 books.
Recommending highest 20 predicted ratings books not already rated.


### Tope 20 Books that user id 20 has rated

In [25]:
already_rated.head(20)

,user_id,book_id,rating,authors,original_title
54,20,821,5,Nicholas Sparks,The Best of Me
1,20,4913,5,Debbie Macomber,The Shop on Blossom Street
30,20,308,5,Sophie Kinsella,Can You Keep a Secret?
27,20,1078,5,David Allen,Getting Things Done: How To Achieve Stress-fre...
64,20,1260,5,"James Patterson, Peter de Jonge",The Beach House
24,20,31,5,Kathryn Stockett,The Help
21,20,273,5,Lisa See,Snow Flower and the Secret Fan
61,20,3455,5,James Patterson,The Lake House
19,20,674,5,James Patterson,Suzanne's Diary for Nicholas
18,20,1103,5,James Patterson,Sam's Letters To Jennifer


### Top 20 Book that user id 20 will enjoy reading

In [26]:
predictions

,book_id,authors,original_title
41,44,Nicholas Sparks,The Notebook
2669,2737,James Patterson,The Big Bad Wolf
2217,2277,James Patterson,Cross
3146,3223,James Patterson,London Bridges
36,38,Audrey Niffenegger,The Time Traveler's Wife
307,328,Jodi Picoult,Nineteen Minutes
2453,2519,James Patterson,When the Wind Blows
3940,4022,James Patterson,Cross Country
3748,3829,James Patterson,"I, Alex Cross"
144,156,Emily Giffin,Something Borrowed


### Model Evaluation 

In [27]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD, evaluate

# Load Reader library
reader = Reader()

# Load ratings dataset with Dataset library
data = Dataset.load_from_df(ratings[['user_id', 'book_id', 'rating']], reader)

# Split the dataset for 5-fold evaluation
data.split(n_folds=5)

### We will get the mean of RMSE of 0.82972

In [28]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
evaluate(svd, data, measures=['RMSE'])

C:\Users\Ayham\Anaconda3\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\Ayham\Anaconda3\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.8297
------------
Fold 2
RMSE: 0.8303
------------
Fold 3
RMSE: 0.8303
------------
Fold 4
RMSE: 0.8301
------------
Fold 5
RMSE: 0.8301
------------
------------
Mean RMSE: 0.8301
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.8297225356269575,
                             0.8303300189269696,
                             0.8303261054225649,
                             0.8301270536439884,
                             0.830127442968465]})

In [29]:
trainset = data.build_full_trainset()
svd.train(trainset)

C:\Users\Ayham\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


### We will pick user id 20 and check the ratings he has given: 

In [30]:
ratings[ratings['user_id'] == 20]

,user_id,book_id,rating
2168421,20,761,4
2168422,20,4913,5
2168423,20,8689,5
2168424,20,6528,4
2168425,20,9168,4
2168426,20,9827,4
2168429,20,152,5
2168430,20,107,4
2168433,20,169,5
2168434,20,286,4


### We will use SVD to predict the rating that user id 20 with book id 1991:

In [34]:
svd.predict(20, 1991)

Prediction(uid=20, iid=1991, r_ui=None, est=4.549633072946882, details={'was_impossible': False})

### As you can see the prediction is 4.54. The recommender system works purely on the basis of an assigned book id and tries to predict ratings based on how the other users have predicted the book.